# Selecting the ultimate Six Nations fantasy team with Python and linear programming

- A few years out from following rugby, a few years into maths
- Invited to a fantasy league with friends from uni
- Geraint's blogpost: http://www.geraintianpalmer.org.uk/2018/05/29/pokemon-team-pulp/
- Python Rugby API: https://github.com/walsh06/python_rugby

### What makes a team?

- 15 players, plus 3 subs (half points)
- No more than 4 players from any nation
- At least one of each position
- No more than:
  - 2 full-back, fly-half, scrum-half, hooker
  - 4 winger, centre, second row, prop
  - 6 flanker
- One captain (double points + 20 bonus)
- One supersub (triple points if not a first-team player)
- Total cost under budget (240pts to start)

### How is a team's quality measured?

Each player earns points by playing in real life as part of their team and on their own. These points are summed across all players in the fantasy team to give the team score. Points are scored as follows:

**Team points**

NB: team points are given in proportion to the amount of time the player spends on the pitch.

- Home win: 12 points
- Home draw: 4 points
- Home defeat: 1 point
- Away win: 18 points
- Away draw: 10 points
- Away defeat: 3 points
- Margin of victory / defeat (+ of 0 .5 points per point difference): + or - 0.5 points

**Individual points**

- Tackles: 1 point per tackle
- Defenders Beaten: 2 points per defenders beaten
- Carried meters: 0.2 points per carried meter with the ball
- Official Guinness Six Nations Man of the Match: 15 points
- Try:
  - Forwards: 18 points
  - Backs: 15 points
- Try Conversion: 3 points
- Penalty kick: 3 points
- Drop goal: 6 points
- Yellow cards: -5 points
- Red cards: -8 points

In [1]:
import numpy as np
import pandas as pd

from python_rugby.match import MatchList
from python_rugby.player import PlayerList
from python_rugby import rugby_stats as rstats


In [2]:
england_names = [
    "Jack Clifford",
    "Dan Cole",
    "Luke Cowan-Dickie",
    "Tom Curry",
    "Ben Earl",
    "Ellis Genge",
    "Jamie George",
    "Nathan Hughes",
    "Maro Itoje",
    "George Kruis",
    "Joe Launchbury",
    "Courtney Lawes",
    "Ben Moon",
    "Brad Shields",
    "Kyle Sinckler",
    "Jack Singleton",
    "Billy Vunipola",
    "Mako Vunipola",
    "Harry Williams",
    "Mark Wilson",
    "Chris Ashton",
    "Mike Brown",
    "Joe Cokanasiga",
    "Elliot Daly",
    "Ollie Devoto",
    "Owen Farrell",
    "George Ford",
    "Jonny May",
    "Jack Nowell",
    "Dan Robson",
    "Henry Slade",
    "Ben Te'o",
    "Ollie Thorley",
    "Manu Tuilagi",
    "Ben Youngs",
]

england_positions = [
    "Lf",
    "Pr",
    "Hk",
    "Lf",
    "Lf",
    "Pr",
    "Hk",
    "Lf",
    "L",
    "L",
    "L",
    "L",
    "Pr",
    "Lf",
    "Pr",
    "Hk",
    "Lf",
    "Pr",
    "Pr",
    "Lf",
    "Wg",
    "Fb",
    "Wg",
    "C",
    "C",
    "Fh",
    "Fh",
    "Wg",
    "Wg",
    "Sh",
    "C",
    "C",
    "Wg",
    "C",
    "Sh",
]

england_values = [
    10,
    10,
    11,
    9,
    11,
    9,
    14,
    12,
    15,
    14,
    15,
    14,
    11,
    9,
    12,
    10,
    15,
    13,
    12,
    10,
    15,
    12,
    11,
    14,
    10,
    15,
    15,
    15,
    13,
    9,
    12,
    14,
    10,
    14,
    13,
]

england = pd.DataFrame(
    {"name": england_names, "position": england_positions, "value": england_values}
)


In [3]:
england_matches = MatchList.create_for_team("england")


In [4]:
player_lists = {}
for match in england_matches:
    for player in match.players["england"]:
        try:
            player_lists[player.name] += [player]
        except KeyError:
            player_lists[player.name] = [player]


In [5]:
def get_team_stats(matches, name):

    home_wins, home_draws, home_defeats = 0, 0, 0
    away_wins, away_draws, away_defeats = 0, 0, 0

    margins = 0

    for match in matches:
        home, away = match.match_stats["points"].values()

        if match.home_team["name"] == name:

            if home > away:
                home_wins += 1
            elif home == away:
                home_draws += 1
            else:
                home_defeats += 1
            margins += home - away

        else:

            if away > home:
                away_wins += 1
            elif away == home:
                away_draws += 1
            else:
                away_defeats += 1
            margins += away - home

    return (
        team_stat / len(matches)
        for team_stat in (
            home_wins,
            home_draws,
            home_defeats,
            away_wins,
            away_draws,
            away_defeats,
            margins,
        )
    )


In [6]:
def get_expected_team_points(matches, name):

    hwin, hdraw, hdefeat, awin, adraw, adefeat, margin = get_team_stats(matches, name)

    points = (
        12 * hwin
        + 4 * hdraw
        + hdefeat
        + 18 * awin
        + 10 * adraw
        + 3 * adefeat
        + 0.5 * round(margin)
    )

    return points


In [12]:
def get_player_stats(names, player_lists, stat):

    stats = []

    for name in names:

        vals = []
        if name in player_lists.keys():
            for player in player_lists[name]:
                if player.minutes_played > 0:
                    if stat == "time played":
                        vals.append(player.minutes_played / 80)
                    elif player.get_stat(stat) is not None:
                        vals.append(player.get_stat_per_eighty(stat))

        if vals:
            avg = np.mean(vals)
        else:
            avg = np.nan

        stats.append(avg)

    return stats


In [13]:
def get_expected_player_points(player):
    """ Expected points per 80 minutes playing. """

    try_multiplier = 18 if player["position"] in ["Hk", "L", "Lf", "Pr"] else 15
    points = (
        player["tackles"]
        + 2 * player["defenders beaten"]
        + 0.2 * player["meters run"]
        + try_multiplier * player["tries"]
        + 3 * player["conversion goals"]
        + 3 * player["penalty goals"]
        + 6 * player["drop goals converted"]
        - 5 * player["yellow cards"]
        - 8 * player["red cards"]
    )

    return points


In [14]:
stat_names = [
    "conversion goals",
    "defenders beaten",
    "drop goals converted",
    "meters run",
    "penalty goals",
    "red cards",
    "tackles",
    "time played",
    "tries",
    "yellow cards",
]

for stat in stat_names:
    england[stat] = get_player_stats(england["name"], player_lists, stat)


In [15]:
england["EPP"] = [
    get_expected_player_points(player) for _, player in england.iterrows()
]

england["ETP"] = get_expected_team_points(england_matches, "england")

england["expected points"] = england["EPP"] + england["ETP"] * england["time played"]


In [16]:
england = (
    england.sort_values("expected points", ascending=False)
    .reset_index(drop=True)
    .dropna()
)
england


,name,position,value,conversion goals,defenders beaten,drop goals converted,meters run,penalty goals,red cards,tackles,time played,tries,yellow cards,EPP,ETP,expected points
0,Joe Cokanasiga,Wg,11,0.000000,4.500000,0.000000,76.000000,0.000000,0.0,2.500000,1.000000,1.000000,0.000000,41.700000,16.47619,58.176190
1,Jack Nowell,Wg,13,0.000000,3.829387,0.000000,46.056669,0.000000,0.0,2.736401,0.759896,0.661550,0.000000,29.529755,16.47619,42.049943
2,Mike Brown,Fb,12,0.000000,2.680298,0.000000,81.093976,0.000000,0.0,2.266460,0.895486,0.229074,0.000000,27.281959,16.47619,42.036159
3,Owen Farrell,Fh,15,1.765021,0.915090,0.031250,16.959786,2.205100,0.0,5.939255,0.964063,0.190034,0.000000,26.109763,16.47619,41.993840
4,Jonny May,Wg,15,0.000000,2.839243,0.000000,62.073425,0.000000,0.0,2.272827,0.896000,0.400000,0.055556,26.088221,16.47619,40.850888
5,Manu Tuilagi,C,14,0.000000,7.272727,0.000000,20.000000,0.000000,0.0,1.753247,0.580357,0.582418,0.000000,29.034965,16.47619,38.597040
6,Nathan Hughes,Lf,12,0.000000,2.529870,0.000000,63.178771,0.000000,0.0,7.448556,0.652500,0.075117,0.136752,25.812402,16.47619,36.563117
7,Billy Vunipola,Lf,15,0.000000,2.750678,0.000000,34.476621,0.000000,0.0,3.942976,0.871324,0.285714,0.000000,21.482512,16.47619,35.838604
8,Elliot Daly,C,14,0.000000,1.736842,0.000000,37.388889,0.263158,0.0,2.684211,0.859868,0.473684,0.000000,21.530409,16.47619,35.697765
9,Maro Itoje,L,15,0.000000,0.869565,0.000000,15.791631,0.000000,0.0,11.538164,0.960870,0.043478,0.043478,17.000838,16.47619,32.832308
